In [ ]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [ ]:
# Importing relevant libraries
import cvxpy as cp
import numpy as np

In [ ]:
# Loading data
from solara.constants import PROJECT_PATH
load_data = np.loadtxt(PROJECT_PATH + "/data/solar_trace_data/load_5796.txt", delimiter=",")
pv_data = np.loadtxt(PROJECT_PATH + "/data/solar_trace_data/PV_5796.txt", delimiter=",")

In [ ]:
# Setting all the variables

# Given variables

# Battery
B = 10 # Capacity of ESD (kWh)
MD = 0.1 # ESD maximum discharge and charge fractions
MC = 0.9 # See above
alpha_c = 0.33 # Charge (discharge) rate limits per unit of storage (kW/kWh)
alpha_d = 1.67 # See above
eta_c = 0.85 # Charge (discharge) efficiency. Both are≤1
eta_d = 1 # # See above
U = B*MD

# Grid
pi_b = 0.14 #0.14 # Base price per unit of energy purchased ($/kWh)
pi_d = 0.86 # Demand price penalty per unit of energy purchased with power demand exceeding Γ($/kWh)
Gamma = 1.00 # np.percentile(load_data, 80) # Threshold above which the demand price is paid (kW)
p_bar = 0.12 # Price per unit of energy sold at time t ($/kWh)

# Basic
T_u = 1 # Time slot duration
T_h = 24 # Time horizon (hours)


# Given variables from data set
num_timesteps = T_h
start = 24*12
P_L = load_data[start:start+num_timesteps] #np.random.randn(num_timesteps) # Load at time t (kW)
P_S = pv_data[start:start+num_timesteps] #np.random.randn(num_timesteps) # Power generated by solar panels at timet(kW)


# Variables that being optimised over
P_dir = cp.Variable(num_timesteps) # Power flowing directly from PV and grid to meet the load or be sold at time t (kW)
P_c = cp.Variable(num_timesteps) # Power used to charge the ESD at time t (kW)
P_d = cp.Variable(num_timesteps) # Power from the ESD at time t (kW)
P_g = cp.Variable(num_timesteps) # Power drawn from the grid at time t (kW)
P_sell = cp.Variable(num_timesteps) # Power sold to the grid at timet(kW)
P_over = cp.Variable(num_timesteps) #  Purchased power that exceeds Γ at time t (not in notation table)
I = cp.Variable(num_timesteps, boolean=True) # Boolean complies with contraint Eq (20)

# Implicitly defined variable (not in paper in "given" or "optimized over" set of variables)
E_ESD = cp.Variable(num_timesteps+1) # the  energy  content  of  the  ESD  at  the  beginning  of  interval t

In [ ]:
eta_d = 1 / 0.9  # taking reciprocal so that we don't divide by eta_d
eta_c = 0.9942

In [ ]:
base_constraints = [
               0 <= P_g, # from Equation (13)
               0 <= P_dir,
               0 <= P_sell,
               0 <= P_c, # Eq (18)
               0 <= P_d, # Eq  (19)
               0 <= P_over, # Eq (23)
               
               # Power flow
               P_dir + P_d == P_L + P_sell, # from Equation (14)
               0 <= P_c + P_dir, # Eq (17)
               P_c + P_dir <= P_S + P_g, # Eq (17)
    ]
               
battery_constraints = [
               E_ESD[0] == 1, # Eq (15)
               E_ESD[1:] == E_ESD[:-1] + eta_c*P_c*T_u - (1/eta_d) * P_d * T_u, # Eq (16)
               P_d <= (1 - I) * B * alpha_d, # Eq  (19)
               P_c <= I * B * alpha_c, # Eq (18)
               B*MD <= E_ESD, # Eq (21)
               E_ESD <= B*MC, # Eq (21)
    ]
               
grid_constraints = [
               P_g - Gamma <= P_over, # Eq (24)
               P_sell == 0, # stopping selling to the grid
    ]

In [ ]:
# Battery variables
size = 10
kWh_per_cell = 0.011284
num_cells = size / kWh_per_cell

# parameters specified for an LNMC cell with operating range of 1 C
# charging and discharging
nominal_voltage_c = 3.8793
nominal_voltage_d = 3.5967
u1 = 0.1920
v1_bar = 0.0
u2 = -0.4865
v2_bar = kWh_per_cell * num_cells
eta_d = 1 / 0.9  # taking reciprocal so that we don't divide by eta_d
eta_c = 0.9942
alpha_bar_d = (
    v2_bar * 1
)  # the 1 indicates the maximum discharging C-rate
alpha_bar_c = (
    v2_bar * 1
)  # the 1 indicates the maximum charging C-rate

battery_constraints = [
    E_ESD[0] == 0,
    E_ESD[1:] == E_ESD[:-1] + eta_c*P_c*T_u - eta_d * P_d * T_u,
    E_ESD >= 0,
    P_d <= alpha_bar_d,
    P_c <= alpha_bar_c, #equation (5)
    u1 * ((P_d)/nominal_voltage_d) + v1_bar <= E_ESD[1:], # equation (4)
    u2 * ((P_c)/nominal_voltage_c) + v2_bar >= E_ESD[1:], # equation (4)
    
]

In [ ]:
constraints = base_constraints + battery_constraints + grid_constraints

In [ ]:
objective = cp.Minimize(cp.sum(pi_b*P_g + pi_d*P_over - cp.multiply(p_bar,P_sell)))

In [ ]:
prob = cp.Problem(objective, constraints)

In [ ]:
result = prob.solve(verbose=True)

In [ ]:
result

In [ ]:
charging_power = P_c.value - P_d.value

episode_data = {
    'battery_cont': E_ESD.value,
    'charging_power': charging_power,
    'actions': charging_power / 10,
    'load': P_L,
    'pv_gen': P_S,
    'rewards': - (pi_b*P_g.value + pi_d*P_over.value),
    'cost': pi_b*P_g.value + pi_d*P_over.value,
    'power_diff': np.zeros(24),
}

In [ ]:
import solara.utils.rllib
import solara.plot.widgets

initial_visibility = ['load','pv_gen','energy_cont','net_load',
                      'charging_power','cost','price_threshold',
                      'actions']
initial_visibility = ['energy_cont', 'pv_gen', 'actions', 'charging_power', 'energy_cont']

solara.plot.widgets.InteractiveEpisodes([episode_data], initial_visibility=initial_visibility)

In [ ]:
import solara.envs.components.solar
import solara.envs.components.load
import solara.envs.components.grid
import solara.envs.components.battery
import solara.envs.battery_control
import solara.utils.logging
from solara.constants import PROJECT_PATH


def battery_env_creator(env_config=None):
    """Create a battery control environment."""
    
    PV_DATA_PATH = PROJECT_PATH + "/data/solar_trace_data/PV_5796.txt"
    LOAD_DATA_PATH = PROJECT_PATH + "/data/solar_trace_data/load_5796.txt"

    # Setting up components of environment
    battery_model = solara.envs.components.battery.LithiumIonBattery(size=10, 
                                                                     chemistry="NMC", 
                                                                     time_step_len=1)
    pv_model = solara.envs.components.solar.DataPV(data_path=PV_DATA_PATH,
                                                   fixed_sample_num=12)
    load_model = solara.envs.components.load.DataLoad(data_path=LOAD_DATA_PATH,
                                                      fixed_sample_num=12)
    grid_model = solara.envs.components.grid.PeakGrid(peak_threshold=1.0)

    # Fixing load and PV trace to single sample
    episode_num = 12
    load_model.fix_start(episode_num)
    pv_model.fix_start(episode_num)

    env = solara.envs.battery_control.BatteryControlEnv(
        battery = battery_model,
        pv_system = pv_model,
        grid = grid_model,
        load = load_model,
        infeasible_control_penalty=True,
        grid_charging=True,
        logging_level = "WARNING",
    )
    
    return env

env = battery_env_creator()

solara.plot.widgets.InteractiveEpisodes([episode_data], 
                                        initial_visibility=initial_visibility, 
                                        manual_mode=True, 
                                        manual_start_actions=episode_data["actions"],
                                        env=env)

In [ ]:
# 0.005 are caused by the inaccuracy of the linear approximation

In [ ]:
#'battery_cont': 
np.array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.056, 0.471, 0.958, 1.577, 2.185, 2.715, 3.058, 3.086,
       2.933, 2.226, 1.182, 0.686, 0.518, 0.271, 0.019])

In [ ]:
E_ESD.value

In [ ]:
P_d.value

In [ ]:
P_c.value

In [ ]:
u1 * ((P_c.value - P_d.value)/nominal_voltage_d) + v1_bar